# Missing values and imputation


## Missing values

**Source of missing values:**

* Data Extraction: It is possible that there are problems with extraction process. In such cases, we should double-check for correct data with data guardians. Some hashing procedures can also be used to make sure that data extraction is correct. Errors at data extraction stage are typically easy to find and can be corrected easily as well.
* Data collection: These errors occur at time of data collection and are harder to correct. They can be categorized in four types:
>* Missing completely at random: This is a case when the probability of missing variable is same for all observations. For example: respondents of data collection process decide that they will declare their earning after tossing a fair coin. If an head occurs, respondent declares his / her earnings & vice versa. Here each observation has equal chance of missing value.
>* Missing at random: This is a case when variable is missing at random and missing ratio varies for different values / level of other input variables. For example: We are collecting data for age and female has higher missing value compare to male.
>* Missing that depends on unobserved predictors: This is a case when the missing values are not random and are related to the unobserved input variable. For example: In a medical study, if a particular diagnostic causes discomfort, then there is higher chance of drop out from the study. This missing value is not at random unless we have included “discomfort” as an input variable for all patients.
>* Missing that depends on the missing value itself: This is a case when the probability of missing value is directly correlated with missing value itself. For example: People with higher or lower income are likely to provide non-response to their earning.

We are going to use the Titanic dataset again in this tutorial to explore missing data.

In [ ]:
#import library 
import pandas as pd
import numpy as np

In [ ]:
#load the data 
df1 = pd.read_csv('titanic passenger list.csv') 

We would normally check some data basics before analysis, such as 
* length (rows)
* width (or columns)
* column types
* basic stats
* head & tail of data

We will skip this step in this tutorial.

## Task 1 Check missing values 


** info()**
* tells us total number of records
* gives us details on datatype, 'object' here means string, the others are int and float
* also include see 'non-null' observations for each data column

** describe() **by default tabulates numeric data, which report on:
* descriptive statistics 
* number of non-null observations in "count"
* with "include='all'" will include all columns and "include=['O']" will include objects

**.isnull().sum()***
* will return sum of missing values for each column



There are 1309 x 14 records and there are many missing values in "age","fare" (one record) "cabin", "boat" "body" and "home.dest".

In [ ]:
 # 'O' for Objects


#### Which of these are potentially imputatable i.e. we can derive values from existing data?


## Task 2 investigate with missing values for ages

Titanic data is commonly used for predictive analytics, e.g. remove the 'survived' column and feed the other data into an algorithm to see if there are any predictors for survival, pclass was a factor, also gender, age another (and if you ever go on a cruise ship change your name to "Womenandchildren").

Age are mostly missing. Should we just delete or do something basic like:  

meanAge = np.mean(df1.age)

df1.age = df1.age.fillna(meanAge) 

### Discussion:

* **What's wrong with this approach?**

* **What is the average age anyway?**

*  **What is the average age for males, for females? Is that an improvement?**


In [ ]:
# age - we know there are some missing, let's dig deeper


In [ ]:
# mostly younger, 20s to 30s


Notice the 'nan' above, it doesn't occur in counts() 

### Discussion:
*  How many 'NaN' values for age are there?
*  How many 0?
*  How many < 1?
*  What does 'NaN' mean?
*  Is NaN == NaN true in Python?

#### What does age = 0.17 mean?

In [ ]:
%matplotlib inline

In [ ]:
df1['age'].hist()

In [ ]:
sum(df1['age'].isnull())
# 263 passengers have no age recorded.

### Let's look at other factors, e.g. names can give a clue because most passengers seem to have a title (e.g. 'Mrs')

### Discussion:

* Can we assume that 'Mrs' implies married (implies not a child)?

* What about 'Miss' or 'Ms'?


In [ ]:
# Look into titles, e.g. 'Mrs' implies married (implies not child)
def name_extract(word):
     return word.split(',')[1].split('.')[0].strip()
    
# because names are in this format:
# Allison, Master. Hudson Trevor
# we can split on ','
# then '.'

In [ ]:
# testing, apply the method to the data
temp =  
# check unique values
temp['Title'].unique()

In [ ]:
# a couple of strange ones but most of the standard titles are there


In [ ]:
# did we miss any?


### That worked really well, we have titles for everyone and there are only a few strange ones:

Someone look up 'Jonkheer' (and Dona - Spanish?)

#### So what can we say about 'Master'? Usually refers to boys?

#### Dr, Rev, Col, Major, Capt, Sir, Don, would all be adults? 

Let's go ahead and apply this transformation:

In [ ]:
df2 = df1 # copy then insert new column
df2['Title'] = df1['name'].apply(name_extract)
df2.head() # title at far right

In [ ]:
# just check (again) we got most of them


#### First let's check distribution of missing values across different titles

### We will first investigate missing values in doctors

In [ ]:
df2[df2['Title'] == "Dr"]

Drs are mostly older (except for Dr Pain - hah)

#### What's the average age of Drs?

The average age for all passengers is 29, for Drs it's 43, an improvement in fidelity?


no missing values so move on..

#### Your turn, look at 'Maj' and any other titles that suggest age


## Missing values: ages for boys

It seems, in this era (pre Jackie Chan), Master did mean boys 

In [ ]:
df2[df2['Title'] == "Master"] # how many?

In [ ]:
#there are a lot, 61, use describe()
df2[df2['Title'] == "Master"].describe() # min age is 0.33 (4 months?), max is 14.5, mean is 5.5

In [ ]:
df2[df2['Title'] == "Master"].mean()

### Discussion

* How many boys don't have an age?
* What does describe() (above) tell us about pclass, sibsp, parch?
* Do any children not have any siblings or parents travelling with them?
* Any other factors that can help determine (or impute) age?



## Task 3 Mean imputation of age by title 

In this task you need to calculate the mean age for each title group and use the imputed values to replace the missing values in the corresponding group. 

In [ ]:
# this seems too easy, is it right? 
